In [ ]:
import numpy as np
from PIL import Image
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

#Augmentation.ipynb

In [ ]:
#THIS BLOCK IS IMPORTANT


def importClasses():
    classPath = "./tiny-imagenet-200/wnids.txt"
    return open(classPath).read().splitlines()

trainClasses = importClasses()

def createDict(trainClasses):
    #Mapping from class name to class Number
    classDict = {}
    for i in range(len(trainClasses)):
        classDict[trainClasses[i]] = i
    return classDict

classNumDict = createDict(trainClasses)

def getMiddlePatch(path):
    try:
        img = Image.open(path)
    except IOError:
        return
    img = img.convert("RGB")
    img = np.asarray(img)
    return img[3:59, 3:59, :]
    

def pcaAug(original_image):
    #Takes in an image as a np array and returns the pca augmented image as a np array
    renorm_image = np.reshape(original_image,(original_image.shape[0]*original_image.shape[1],3))

    renorm_image = renorm_image.astype('float32')
    renorm_image -= np.mean(renorm_image, axis=0)
    renorm_image /= np.std(renorm_image, axis=0)

    cov = np.cov(renorm_image, rowvar=False)

    lambdas, p = np.linalg.eig(cov)
    alphas = np.random.normal(0, 0.1, 3)

    delta = np.dot(p, alphas*lambdas)

    delta = (delta).astype('int8')

    pca_color_image = np.maximum(np.minimum(original_image + delta, 255), 0).astype('uint8')
    return pca_color_image


def transToOneHot(numClass, dims = 200):
    vec = np.zeros(dims)
    vec[numClass] = 1
    return vec

def generateImgs(path, outClassNum):
    #Takes in path of Image and generates 5 patches and their mirror images and returns them as a list of np arrays and an array of one hot encoded output
    try:
        img = Image.open(path)
    except IOError:
        return
    img = img.convert("RGB")
    img = np.asarray(img)
    arr = [img[0:56, 0:56, :], img[8:, 0:56, :], img[0:56, 8:, :], img[8:, 8:, :], img[3:59, 3:59, :]]
    arr = arr + [np.flip(arr[0], 1),np.flip(arr[1], 1), np.flip(arr[2], 1), np.flip(arr[3], 1), np.flip(arr[4], 1)]
    pcaArr = []
    for i in range(len(arr)):
        pcaArr.append(pcaAug(arr[i]))
    inputArr = arr + pcaArr
    oneHot = transToOneHot(outClassNum, dims = 200)
    outArr = []
    for i in range(len(inputArr)):
        outArr.append(oneHot)
    return inputArr, outArr

def getTrainingSample():
    #Returns one batch of training samples using any 5 random images. Returns a batch of 100
    inputArray = []
    outputArray = []
    for i in range(5):
        outClassNum = np.random.randint(len(trainClasses))
        path = "./tiny-imagenet-200/train/" + str(trainClasses[outClassNum]) + "/images/" + str(trainClasses[outClassNum]) + "_" + str(np.random.randint(100)) + ".JPEG"
        inArr, outArr = generateImgs(path, outClassNum)
        inputArray = inputArray + inArr
        outputArray = outputArray + outArr
    #temp = np.array(100,3)
    #temp[:, 0] = inputArray
    #temp[:,1] = outputArray
    #temp[:,2] = range(100)
    #np.random.shuffle(temp)
    #inputArray = temp[:,0]
    #outputArray = temp[:,1]
    return inputArray, outputArray

def getOutputArray():
    outputArray = []
    outputTextPath = "./tiny-imagenet-200/val/val_annotations.txt"
    textFile = open(outputTextPath).read().split()
    for i in range(1000):
        outputArray.append(transToOneHot(classNumDict[textFile[6*i+1]], dims = 200))
    return outputArray

def getTestSample():
    inputArray = []
    outputArray = getOutputArray()
    
    
    for i in range(1000):
        path = "./tiny-imagenet-200/val/images/val_" + str(i) + ".JPEG"
        img = getMiddlePatch(path)
        inputArray.append(img)
        
    return inputArray, outputArray

#imagenet.ipynb

In [ ]:
image_size = 56 #We should use 56 instead, using 64x64 images

input_images = tf.placeholder(tf.float32, shape= [None, image_size, image_size,3], name = "input_images")
kernel = tf.Variable(tf.truncated_normal([11,11,3,96], dtype=tf.float32, stddev=1e-2), name="conv1_weights")

conv = tf.nn.conv2d(input_images, kernel, [1,4,4,1], padding="SAME") #[1,4,4,1] === [1,stride,stride,1] see documentation
bias = tf.Variable(tf.truncated_normal([96]), name="conv1_bias")
conv_with_bias = tf.nn.bias_add(conv, bias)
conv1 = tf.nn.relu(conv_with_bias, name="conv1")
lrn1 = tf.nn.lrn(conv1, alpha = 1e-4, beta=0.75, depth_radius=5, bias=2.0) #LRN = Local Response Normalisation..Depth Radius is most probably n, need to verify
pooled_conv1 = tf.nn.max_pool(lrn1, ksize=[1,3,3,1], strides=[1,2,2,1], padding="SAME", name="pool1")

In [ ]:
kernel = tf.Variable(tf.truncated_normal([5,5,96,256], dtype=tf.float32, stddev=1e-2), name="conv2_weights")
conv = tf.nn.conv2d(pooled_conv1, kernel, [1, 4, 4, 1], padding="SAME")
bias = tf.Variable(tf.ones([256]), name="conv2_bias")
conv_with_bias = tf.nn.bias_add(conv, bias)
conv2 = tf.nn.relu(conv_with_bias, name="conv2")
lrn2 = tf.nn.lrn(conv2,
                 alpha=1e-4,
                 beta=0.75,
                 depth_radius=5,
                 bias=2.0)

pooled_conv2 = tf.nn.max_pool(lrn2,
                              ksize=[1, 3, 3, 1],
                              strides=[1, 2, 2, 1],
                              padding="SAME",
                              name="pool2")


In [ ]:
kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 384],
                                         dtype=tf.float32,
                                         stddev=1e-2),
                     name="conv3_weights")
conv = tf.nn.conv2d(pooled_conv2, kernel, [1, 1, 1, 1], padding="SAME")
bias = tf.Variable(tf.truncated_normal([384]), name="conv3_bias")
conv_with_bias = tf.nn.bias_add(conv, bias)
conv3 = tf.nn.relu(conv_with_bias, name="conv3")

In [ ]:
kernel = tf.Variable(tf.truncated_normal([3, 3, 384, 384],
                                         dtype=tf.float32,
                                         stddev=1e-2),
                     name="conv4_weights")
conv = tf.nn.conv2d(conv3, kernel, [1, 1, 1, 1], padding="SAME")
bias = tf.Variable(tf.ones([384]), name="conv4_bias")
conv_with_bias = tf.nn.bias_add(conv, bias)
conv4 = tf.nn.relu(conv_with_bias, name="conv4")

In [ ]:
kernel = tf.Variable(tf.truncated_normal([3, 3, 384, 256],
                                         dtype=tf.float32,
                                         stddev=1e-2),
                     name="conv5_weights")
conv = tf.nn.conv2d(conv4, kernel, [1, 1, 1, 1], padding="SAME")
bias = tf.Variable(tf.ones([256]), name="conv5_bias")
conv_with_bias = tf.nn.bias_add(conv, bias)
conv5 = tf.nn.relu(conv_with_bias, name="conv5")

In [ ]:
fc_size = 256
conv5 = tf.layers.flatten(conv5)
weights = tf.Variable(tf.truncated_normal([fc_size, fc_size]), name = "fc1_weights")
bias = tf.Variable(tf.ones([fc_size]), name="fc1_bias")
fc1 = tf.matmul(conv5, weights) + bias
fc1 = tf.nn.dropout(fc1, keep_prob = 0.5) #Check this line, might cause problem
fc1 = tf.nn.relu(fc1, name="fc1")

In [ ]:
weights = tf.Variable(tf.truncated_normal([fc_size, fc_size]), name="fc2_weights")
bias = tf.Variable(tf.ones([fc_size]), name="fc2_bias")
fc2 = tf.matmul(fc1, weights) + bias
fc2 = tf.nn.dropout(fc2, keep_prob = 0.5) #Check this line, might cause problem
fc2 = tf.nn.relu(fc2, name="fc2")

In [ ]:
n_classes = 200
weights = tf.Variable(tf.zeros([fc_size, n_classes]), name="output_weights")
bias = tf.Variable(tf.ones([n_classes]), name="output_bias")
out = tf.matmul(fc2, weights) + bias
out = tf.nn.softmax(out)

#training.ipynb

In [ ]:
# all the parameters
learning_rate = 0.01
momentum = 0.9
weight_decay = 0.0005
no_of_epochs = 5
total_batches = 5

y = tf.placeholder(tf.float32, [None, n_classes])
var_list = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

# cost function and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y, logits = out))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)


# optimizer = tfa.optimizers.weight_decay_optimizers.SGDW(
#                 learning_rate=learning_rate, 
#                 momentum=momentum, 
#                 weight_decay=weight_decay, 
#                 nesterov=True, name='SGDW').minimize(cost, var_list = var_list)


# accuracy functions
top_1 = tf.equal(tf.argmax(out, 1), tf.argmax(y, 1))
top_1_accuracy = tf.reduce_mean(tf.cast(top_1, tf.float32))
top_5 = tf.math.in_top_k(out, tf.argmax(y, 1), 5)
top_5_accuracy = tf.reduce_mean(tf.cast(top_5, tf.float32))


init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    # for plotting
    train_loss = []
    test_loss = []
    train_accuracy_top_1 = []
    train_accuracy_top_5 = []
    test_accuracy_top_1 = []
    test_accuracy_top_5 = []

    inp_test, out_test = getTestSample() # so that after every epoch we test the same dataset

    summary_writer = tf.summary.FileWriter('./Output', sess.graph)

    for epoch in range(no_of_epochs):
        # running the architecture for 1 epoc
        for batch_no in range(total_batches):
            inp_train, out_train = getTrainingSample()
            sess.run([optimizer],
                        feed_dict={
                            input_images: inp_train,
                            y: out_train})
        
        # calculating accuracy and loss after 1 epoc for the last training data
        top_1_acc, top_5_acc, loss = sess.run([top_1_accuracy, top_5_accuracy, cost],
                                                                feed_dict={
                                                                    input_images: inp_train,
                                                                    y: out_train})
        print("Epoch: {}, Top 1 Acc (training): {}, Top 5 Acc (training) = {}, Loss: {} ".format(epoch, top_1_acc, top_5_acc, loss))
            
        # calculating accuracy and loss after 1 epoc for the test data
        test_top_1_acc, test_top_5_acc, valid_loss = sess.run([top_1_accuracy, top_5_accuracy, cost],
                                                                feed_dict={
                                                                    input_images: inp_test,
                                                                    y: out_test})
        print("Top 1 Acc (testing): {}, Top 5 Acc (testing) = {}, Validation Loss: {}".format(test_top_1_acc, test_top_5_acc, valid_loss))

        # storing for plotting
        train_loss.append(loss)
        test_loss.append(valid_loss)
        train_accuracy_top_1.append(top_1_acc)
        train_accuracy_top_5.append(top_5_acc)
        test_accuracy_top_1.append(test_top_1_acc)
        test_accuracy_top_5.append(test_top_5_acc)

    summary_writer.close()

    # plotting training and testing - loss
    plt.figure(0)
    plt.plot(range(len(train_loss)), train_loss, 'b', label='Training loss')
    plt.plot(range(len(train_loss)), test_loss, 'r', label='Test loss')
    plt.title('Training and Test loss')
    plt.xlabel('Epochs ',fontsize=16)
    plt.ylabel('Loss',fontsize=16)
    plt.legend()
    plt.figure()
    plt.show()

    # plotting training and testing - top 1 accuracy
    plt.figure(1)
    plt.plot(range(len(train_loss)), train_accuracy_top_1, 'b', label='Top 1 Training Accuracy')
    plt.plot(range(len(train_loss)), test_accuracy_top_1, 'r', label='Top 1 Test Accuracy')
    plt.title('Top 1 Training and Test Accuracy')
    plt.xlabel('Epochs ',fontsize=16)
    plt.ylabel('Loss',fontsize=16)
    plt.legend()
    plt.figure()
    plt.show()

    # plotting training and testing - top 5 accuracy
    plt.figure(2)
    plt.plot(range(len(train_loss)), train_accuracy_top_5, 'b', label='Top 5 Training Accuracy')
    plt.plot(range(len(train_loss)), test_accuracy_top_5, 'r', label='Top 5 Test Accuracy')
    plt.title('Top 5 Training and Test Accuracy')
    plt.xlabel('Epochs ',fontsize=16)
    plt.ylabel('Loss',fontsize=16)
    plt.legend()
    plt.figure()
    plt.show()

In [ ]:
# Delete Everything after this